### Looking at League and Team API options

In [1]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static.teams import get_teams
import pandas as pd

On the API docs, I found a place where LEAGUE_ID was set to either '20' or '00', so let's try those.

In [2]:
league00 = leaguegamefinder.LeagueGameFinder(league_id_nullable='00')
league20 = leaguegamefinder.LeagueGameFinder(league_id_nullable='20')
print(league00.get_data_frames()[0].TEAM_NAME.unique())
print(league20.get_data_frames()[0].TEAM_NAME.unique())

['Chicago Bulls' 'Portland Trail Blazers' 'Miami Heat' 'Charlotte Hornets'
 'Detroit Pistons' 'Cleveland Cavaliers' 'New Orleans Pelicans'
 'Atlanta Hawks' 'Boston Celtics' 'San Antonio Spurs' 'Denver Nuggets'
 'Los Angeles Lakers' 'Houston Rockets' 'Indiana Pacers' 'Utah Jazz'
 'Philadelphia 76ers' 'Memphis Grizzlies' 'New York Knicks'
 'Toronto Raptors' 'Brooklyn Nets' 'LA Clippers' 'Oklahoma City Thunder'
 'Orlando Magic' 'Phoenix Suns' 'Minnesota Timberwolves'
 'Washington Wizards' 'Dallas Mavericks' 'Golden State Warriors'
 'Milwaukee Bucks' 'Sacramento Kings' 'Guangzhou Long-Lions'
 'Haifa Maccabi Haifa' 'Shanghai Sharks' 'Adelaide 36ers'
 'Melbourne United' 'Flamengo Flamengo' 'Perth Wildcats'
 'New Zealand Breakers' 'Sydney Kings' 'Beijing Ducks' 'Team LeBron'
 'Team Stephen' 'World World' 'USA USA' 'Brisbane Bullets'
 'West NBA All Stars West' 'East NBA All Stars East'
 'East Celebrities East' 'West Celebrities West'
 'Buenos Aires San Lorenzo' 'Barcelona FC Barcelona Lassa'
 

It looks to me like LEAGUE_ID '00' is major leagues teams -- NBA, WNBA, and international -- while LEAGUE_ID '20' is national teams and G-League teams.

That's not a very good level of granularity, it seems like there should be a way to uniquely identify a league.
Maybe there are other 2-digit codes for certain leagues or groups of leagues?
We can easily check them all.

In [3]:
from eventlet.timeout import Timeout
from showprogress import showprogress
leagues = {}
for i in showprogress(range(100), text='Leagues'):
    # Convert the league ID number to a string.
    league_id = str(i).zfill(2)
    teams = (leaguegamefinder.LeagueGameFinder(league_id_nullable=league_id)
             .get_data_frames()[0]
             .TEAM_NAME
             .unique()).tolist()
    leagues[league_id] = teams

[=================                                           ] 29/100

KeyboardInterrupt: 

Every time I ran the above, it hung up -- either on League 10, 33, or 66.
I'll examine just the first 29.

In [4]:
uniqs = {key: pd.Series(leagues[key]).unique().tolist() for key in leagues}
[(key, len(uniqs[key])) for key in uniqs]

[('00', 92),
 ('01', 0),
 ('02', 0),
 ('03', 0),
 ('04', 0),
 ('05', 0),
 ('06', 0),
 ('07', 2),
 ('08', 12),
 ('09', 4),
 ('10', 44),
 ('11', 0),
 ('12', 0),
 ('13', 4),
 ('14', 19),
 ('15', 37),
 ('16', 6),
 ('17', 28),
 ('18', 0),
 ('19', 20),
 ('20', 68),
 ('21', 0),
 ('22', 0),
 ('23', 0),
 ('24', 0),
 ('25', 0),
 ('26', 0),
 ('27', 0),
 ('28', 0)]

The best candidate to be the NBA (at least 30 teams but not too far over) is League 15.
Let's look.

In [5]:
uniqs['15']

['Los Angeles Lakers',
 'Portland Trail Blazers',
 'Cleveland Cavaliers',
 'Memphis Grizzlies',
 'Detroit Pistons',
 'Boston Celtics',
 'Philadelphia 76ers',
 'Toronto Raptors',
 'Houston Rockets',
 'Milwaukee Bucks',
 'Charlotte Hornets',
 'Utah Jazz',
 'Miami Heat',
 'Chicago Bulls',
 'New York Knicks',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Phoenix Suns',
 'Golden State Warriors',
 'Oklahoma City Thunder',
 'Sacramento Kings',
 'Minnesota Timberwolves',
 'Brooklyn Nets',
 'San Antonio Spurs',
 'Orlando Magic',
 'Washington Wizards',
 'Atlanta Hawks',
 'New Orleans Pelicans',
 'LA Clippers',
 'Indiana Pacers',
 'D-League NBA D-League Select',
 'Los Angeles Clippers',
 'Charlotte Bobcats',
 'New Orleans Hornets',
 ' NBA D-League Select',
 'Seattle SuperSonics',
 '  Team China Basketball']

I'd say we found it.
My guess is that the non-NBA teams here (Team China, NBA D-League Select) played against an NBA team at some point so are included in the league's games.

### Teams
The League IDs don't appear reliable for fetching teams.
Luckily, in the nba_api docs I uncovered a `get_teams()` method that appears to be static and simply returns the 30 teams.

In [12]:
teams = get_teams()
print(len(teams))
teams[:5]

30


[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966}]

Looks pretty good.
30 teams, and the info on them is solid.

I was just wondering how teams that are named only after a state (*Minnesota* Timberwolves) were handled and happened on this.

In [13]:
[team for team in teams if team['city'] == team['state']]

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612750,
  'full_name': 'Minnesota Timberwolves',
  'abbreviation': 'MIN',
  'nickname': 'Timberwolves',
  'city': 'Minnesota',
  'state': 'Minnesota',
  'year_founded': 1989},
 {'id': 1610612752,
  'full_name': 'New York Knicks',
  'abbreviation': 'NYK',
  'nickname': 'Knicks',
  'city': 'New York',
  'state': 'New York',
  'year_founded': 1946},
 {'id': 1610612754,
  'full_name': 'Indiana Pacers',
  'abbreviation': 'IND',
  'nickname': 'Pacers',
  'city': 'Indiana',
  'state': 'Indiana',
  'year_founded': 1976},
 {'id': 1610612762,
  'full_name': 'Utah Jazz',
  'abbreviation': 'UTA',
  'nickname': 'Jazz',
  'city': 'Utah',
  'state': 'Utah',
  'year_founded': 1974}]

Really, all we'll need from this to get play-by-play data is team IDs, which are simple to extract.

In [14]:
team_ids = {team['id']: team['full_name'] for team in teams}
print(team_ids)

{1610612737: 'Atlanta Hawks', 1610612738: 'Boston Celtics', 1610612739: 'Cleveland Cavaliers', 1610612740: 'New Orleans Pelicans', 1610612741: 'Chicago Bulls', 1610612742: 'Dallas Mavericks', 1610612743: 'Denver Nuggets', 1610612744: 'Golden State Warriors', 1610612745: 'Houston Rockets', 1610612746: 'Los Angeles Clippers', 1610612747: 'Los Angeles Lakers', 1610612748: 'Miami Heat', 1610612749: 'Milwaukee Bucks', 1610612750: 'Minnesota Timberwolves', 1610612751: 'Brooklyn Nets', 1610612752: 'New York Knicks', 1610612753: 'Orlando Magic', 1610612754: 'Indiana Pacers', 1610612755: 'Philadelphia 76ers', 1610612756: 'Phoenix Suns', 1610612757: 'Portland Trail Blazers', 1610612758: 'Sacramento Kings', 1610612759: 'San Antonio Spurs', 1610612760: 'Oklahoma City Thunder', 1610612761: 'Toronto Raptors', 1610612762: 'Utah Jazz', 1610612763: 'Memphis Grizzlies', 1610612764: 'Washington Wizards', 1610612765: 'Detroit Pistons', 1610612766: 'Charlotte Hornets'}
